In [1]:
%load_ext jupyflink.magics

In [2]:
%flink_gateway_init --debug

debug: True
SessionHandle: 065542a8-dcb1-4229-9ddc-3bde2386a19c


In [3]:
%%flink_gateway_sql_prepare
ADD JAR '/opt/gateway/jars/flink-sql-connector-kafka-1.16.1.jar';

Executing statement: ADD JAR '/opt/gateway/jars/flink-sql-connector-kafka-1.16.1.jar';
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: cd957fdc-ae12-46b8-bcf9-91368be2f8a7
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/cd957fdc-ae12-46b8-bcf9-91368be2f8a7/result/1'}


In [4]:
%%flink_gateway_sql_prepare
drop table if exists attributes;
CREATE TABLE attributes (
  `entityId` STRING,
  `name` STRING,
  `nodeType` STRING,
  `valueType` STRING,
  `type` STRING,
  `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
  `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
  `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
  `deleted` BOOLEAN,
  `ts` TIMESTAMP(3) METADATA FROM 'timestamp',
  WATERMARK FOR ts AS ts
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.attributes',
'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
);
DROP VIEW IF EXISTS `attributes_view`;
CREATE VIEW `attributes_view` AS
SELECT `type`,
`entityId`,
`name`,
`nodeType`,
`valueType`,
`https://uri.etsi.org/ngsi-ld/datasetId`,
`https://uri.etsi.org/ngsi-ld/hasValue`,
`https://uri.etsi.org/ngsi-ld/hasObject`,
`deleted`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `entityId`,`name`, `https://uri.etsi.org/ngsi-ld/datasetId`
ORDER BY ts DESC) AS rownum
FROM `attributes` )
WHERE rownum = 1;


Executing statement: drop table if exists attributes;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: 1f0321cb-a180-4e11-8b49-f309aa70b517
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/1f0321cb-a180-4e11-8b49-f309aa70b517/result/1'}
Executing statement: CREATE TABLE attributes (
  `entityId` STRING,
  `name` STRING,
  `nodeType` STRING,
  `valueType` STRING,
  `type` STRING,
  `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
  `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
  `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
  `deleted` BOOLEAN,
  `ts` TIMESTAMP(3) METADATA FROM 'timestamp',
  WATERMARK 

In [5]:
%%flink_gateway_sql_prepare
drop table if exists ngsild_updates;
CREATE TABLE ngsild_updates (
  `op` STRING,
  `overwriteOrReplace` Boolean,
  `noForward` Boolean,
  `entities` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild-updates',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);

Executing statement: drop table if exists ngsild_updates;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: 031ca0c0-21a8-46ae-afe1-a8e9f8a73d8f
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/031ca0c0-21a8-46ae-afe1-a8e9f8a73d8f/result/1'}
Executing statement: CREATE TABLE ngsild_updates (
  `op` STRING,
  `overwriteOrReplace` Boolean,
  `noForward` Boolean,
  `entities` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild-updates',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);
URL: http://flink-jobmanager:8

In [6]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `propertyChecksTable`;

CREATE TABLE `propertyChecksTable` (
    `targetClass` STRING,
    `propertyPath` STRING,
    `propertyClass` STRING,
    `propertyNodetype` STRING,
    `maxCount` STRING,
    `minCount` STRING,
    `severity` STRING,
    `minExclusive` STRING,
    `maxExclusive` STRING,
    `minInclusive` STRING,
    `maxInclusive` STRING,
    `minLength` STRING,
    `maxLength` STRING,
    `pattern` STRING,
    `ins` STRING, 
    PRIMARY KEY (targetClass,propertyPath) NOT ENFORCED) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True', 
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'propertyChecksTable');


Executing statement: DROP TABLE IF EXISTS `propertyChecksTable`;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: 3baebdaa-ba15-44a8-85de-54a0bfce8197
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/3baebdaa-ba15-44a8-85de-54a0bfce8197/result/1'}
Executing statement: CREATE TABLE `propertyChecksTable` (
    `targetClass` STRING,
    `propertyPath` STRING,
    `propertyClass` STRING,
    `propertyNodetype` STRING,
    `maxCount` STRING,
    `minCount` STRING,
    `severity` STRING,
    `minExclusive` STRING,
    `maxExclusive` STRING,
    `minInclusive` STRING,
    `maxInclusive` STRING,
    `minLength` STR

In [7]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `relationshipChecksTable`;
CREATE TABLE `relationshipChecksTable` (
    `targetClass` STRING,
    `propertyPath` STRING,
    `propertyClass` STRING,
    `maxCount` STRING,
    `minCount` STRING,
    `severity` STRING, 
    PRIMARY KEY (targetClass,propertyPath) NOT ENFORCED) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'relationshipChecksTable');


Executing statement: DROP TABLE IF EXISTS `relationshipChecksTable`;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: d44d42e1-645b-40d9-aebe-fcf0edc904af
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/d44d42e1-645b-40d9-aebe-fcf0edc904af/result/1'}
Executing statement: CREATE TABLE `relationshipChecksTable` (
    `targetClass` STRING,
    `propertyPath` STRING,
    `propertyClass` STRING,
    `maxCount` STRING,
    `minCount` STRING,
    `severity` STRING, 
    PRIMARY KEY (targetClass,propertyPath) NOT ENFORCED) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-o

In [8]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `entity`;
CREATE TABLE `entity` (
    `id` STRING,
    `type` STRING,
    `deleted` BOOLEAN,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp',
    watermark FOR `ts` AS `ts` - INTERVAL '0.001' SECONDS
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'earliest-offset', 
    'topic' = 'iff.ngsild.entities'
);
CREATE VIEW `entity_view` AS SELECT `type`,
 `id`, `deleted`,
 `ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`
ORDER BY ts DESC) AS rownum
FROM `entity` )
WHERE rownum = 1 and `type` IS NOT NULL;

Executing statement: DROP TABLE IF EXISTS `entity`;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: 4534cd96-d7e4-4ec0-a287-088cbd660c02
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/4534cd96-d7e4-4ec0-a287-088cbd660c02/result/1'}
Executing statement: CREATE TABLE `entity` (
    `id` STRING,
    `type` STRING,
    `deleted` BOOLEAN,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp',
    watermark FOR `ts` AS `ts` - INTERVAL '0.001' SECONDS
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstra

In [9]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `rdf`;
CREATE TABLE `rdf` (
`subject` STRING,
`predicate` STRING,
`object` STRING,
`index` INTEGER,
PRIMARY KEY(`subject`,`predicate`,`index`)  NOT ENFORCED
) WITH (
  'connector' = 'upsert-kafka',
  'topic' = 'iff.rdf',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'value.format' = 'json',
  'key.format' = 'json'
);

Executing statement: DROP TABLE IF EXISTS `rdf`;
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: ea1448e2-b9e9-4f9e-aa08-d318d91f428b
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'data': [{'kind': 'INSERT', 'fields': ['OK']}]}, 'resultType': 'PAYLOAD', 'nextResultUri': '/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/operations/ea1448e2-b9e9-4f9e-aa08-d318d91f428b/result/1'}
Executing statement: CREATE TABLE `rdf` (
`subject` STRING,
`predicate` STRING,
`object` STRING,
`index` INTEGER,
PRIMARY KEY(`subject`,`predicate`,`index`)  NOT ENFORCED
) WITH (
  'connector' = 'upsert-kafka',
  'topic' = 'iff.rdf',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'value.format' = 'json',
  'key.format' = 'json'
);
URL: http://flink-jobma

In [10]:
%%flink_gateway_sql_prepare
CREATE TABLE `alerts_bulk` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY<STRING>,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,watermark FOR ts AS ts,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL, 
    PRIMARY KEY (resource,event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092', 
    'topic' = 'iff.alerts.bulk'
);

Executing statement: CREATE TABLE `alerts_bulk` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY<STRING>,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,watermark FOR ts AS ts,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL, 
    PRIMARY KEY (resource,event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092', 
    'topic' = 'iff.alerts.bulk'
);
URL: http://flink-jobmanager:8083/v1/sessions/065542a8-dcb1-4229-9ddc-3bde2386a19c/statements
Result: <Response [200]>
OperationHandle: 34af1157-76d2-452b-8416-df50646ecc54
Status: FINISHED
Status after running: FINISHED
result: {'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'commen